In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
import copy
# import data_loader
import pdb
import torch.nn.functional as F
import re, random, collections
import pickle


import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='2'

In [2]:
class args():

    data_path = "./Datasets/CIFAR100/"
    num_class = 100
    class_per_task = 5
    num_task = 20
    dataset = "cifar100"
    train_batch = 128
    test_batch = 128
    workers = 16
    
    random_classes = False
    validation = 0
    overflow=False
    
    batch_size=128
    lr=0.01
    resume=False
    total_epoch=250
    model_path='ckpt816/cifar100-20'
    
args=args()

In [3]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    
# seed=torch.randint(1,10000,[1])   
set_seed(3473)

In [4]:
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import argparse
# from utils import progress_bar
from torch.optim.lr_scheduler import MultiStepLR
torch.set_printoptions(precision=5,sci_mode=False)

In [5]:
class ConvAdapt(nn.Module):
    def __init__(self, in_channels, out_channels, p):
        super(ConvAdapt, self).__init__()
        # Groupwise Convolution
        self.gwc = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, groups=int(p/gp), bias=True)
        # Pointwise Convolution
        self.pwc = nn.Conv2d(in_channels, out_channels, kernel_size=1, groups=int(p/pt),bias=True)

    def forward(self, x):
        return self.gwc(x) + self.pwc(x)

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.lhc1=ConvAdapt(planes,planes,planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.lhc2=ConvAdapt(planes,planes,planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                ConvAdapt(self.expansion*planes,self.expansion*planes,int(self.expansion*planes)),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.lhc1(self.conv1(x))))
        out = self.bn2(self.lhc2(self.conv2(out)))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=args.class_per_task):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.hc1=ConvAdapt(64,64,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out=self.conv1(x)
        out = F.relu(self.bn1(out))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        feat = out.view(out.size(0), -1)
        out = self.linear(feat)
        return out,feat


def Net():
    return ResNet(BasicBlock, [2,2,2,2])


In [6]:
def save_model(task,acc,model):
    print('Saving..')
    statem = {
        'net': model.state_dict(),
        'acc': acc,
    }
    fname=args.model_path
    if not os.path.isdir(fname):
        os.makedirs(fname)
    torch.save(statem, fname+'/ckpt_task'+str(task)+'.pth')

        
def load_model(task,model):
    fname=args.model_path
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    print(fname+'/ckpt_task'+str(task)+'.pth')
    checkpoint = torch.load(fname+'/ckpt_task'+str(task)+'.pth')
    model.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    return best_acc

In [7]:
def train(train_loader,epoch,task,model,task_model):
    print('\nEpoch: %d' % epoch)
    model.train()
    global best_acc
    train_loss = 0
    correct = 0
    total = 0

    Sloss=0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        targets=targets-task*args.class_per_task # assume sequential split for random  split mapping should me changed
        optimizer.zero_grad()
        outputs,feat_current = model(inputs)
 
        loss = criterion(outputs, targets)
    
        ################################################################################
        ############# this is slow version here we dont save \mu and \sigma of KLD######
        # saving mu and sigma and maximize the distance from the current batch significantly 
        # speedup the model####
        if task>=1:
            for m in range(task):
                with torch.no_grad():
                    task_model[m].eval()
                    out,feat_prev=task_model[m](inputs)
                mean_loss=torch.max(1-torch.dist(feat_current,feat_prev,2)/args.train_batch,
                                    torch.zeros(1).cuda())/task
                loss+=0.05*mean_loss[0]
        ###################################################################################

        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100.*correct/total
    
    print("[Train: ], [%d/%d: ], [Accuracy: %f], [Loss: %f] [LossS: %f] [Lr: %f]" 
          %(epoch,args.total_epoch,acc, train_loss/batch_idx,Sloss/batch_idx,
            optimizer.param_groups[0]['lr']))

In [8]:
def check_task(task,inputs,model,task_model):
    joint_entropy=[]
    with torch.no_grad():
        for m in range(task):
            task_model[m].eval()
            outputs,_=task_model[m](inputs)

            sout=F.softmax(outputs,1)
            entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
            joint_entropy.append(entropy)

        model.eval()
        outputs,_=model(inputs)

        sout=F.softmax(outputs,1)
        entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
        joint_entropy.append(entropy)


    all_entropy=torch.zeros([joint_entropy[0].shape[0],task+1]).cuda()
    for i in range(task+1):
        all_entropy[:,i]=joint_entropy[i]
    ctask=torch.argmin(all_entropy, axis=1)==task
    correct=sum(ctask)
    
    return ctask, correct,all_entropy

    

In [9]:
def test(test_loader,task,model,task_model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    cl_loss=0
    tcorrect=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets1 = inputs.cuda(), targets.cuda()
            targets=targets1-task*args.class_per_task
            if task>0:
                correct_sample,Ncorrect,_=check_task(task,inputs,model,task_model)
                tcorrect+=Ncorrect
                inputs=inputs[correct_sample]
                targets=targets[correct_sample]
            if inputs.shape[0]!=0:
                outputs,_ = model(inputs)
                loss = criterion(outputs, targets)


                test_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()
            total += targets1.size(0)
    if task>0:
        taskC= tcorrect.item()/total
    else:
        taskC=1.0 
    acc = 100.*correct/total
    print("[Test Accuracy: %f], [Loss: %f] [Correct: %f]" %(acc,test_loss/batch_idx,taskC))
    
    
    if acc>=best_acc:
        save_model(task,acc,model)
        best_acc=acc
    return acc

In [10]:
# this false the gradient of the global parameter after the 1st task

def grad_false(model):
    gradf=[0,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133]
    i=0
    for p in model.parameters():
        if i in gradf:
            p.requires_grad=False
        i=i+1

In [11]:
import incremental_dataloader as data
inc_dataset = data.IncrementalDataset(
                                dataset_name=args.dataset,
                                args = args,
                                random_order=args.random_classes,
                                shuffle=True,
                                seed=1,
                                batch_size=args.train_batch,
                                workers=args.workers,
                                validation_split=args.validation,
                                increment=args.class_per_task,
                            )
task_data=[]
for i in range(args.num_task):
    task_info, train_loader, val_loader, test_loader = inc_dataset.new_task()

    task_data.append([train_loader,test_loader])

Files already downloaded and verified
Files already downloaded and verified
0
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
1
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
2
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
3
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
4
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
5
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
6
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
7
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
8
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
9
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
10
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
11
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
12
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
13
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
14
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5

In [ ]:
# groupwise and pointwise group convolutional size
gp=8
pt=16
###############################################

task_model=[]
task_acc=[]
criterion = nn.CrossEntropyLoss()

for task in range(args.num_task):
    if task==0:
        best_acc=0
        print('Training Task :---'+str(task))

        train_loader, test_loader = task_data[task][0],task_data[task][1]
        modelm = Net().cuda()

#         criterion = nn.CrossEntropyLoss()
        mse_loss=nn.MSELoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-3)
        schedulerG = MultiStepLR(optimizer, milestones=[100, 150, 200], gamma=0.1)

        for epoch in range(args.total_epoch):
            train(train_loader,epoch,task,modelm,task_model)
            test(test_loader,task,modelm,task_model)
            schedulerG.step()
        acc1=load_model(task,modelm)
        task_model.append(copy.deepcopy(modelm))


    if task!=0:
        print('Training Task :---'+str(task))
        best_acc=0
        train_loader, test_loader = task_data[task][0],task_data[task][1]
        modelm = Net().cuda()

        acc1=load_model(task-1,modelm)  
        grad_false(modelm)

#         criterion = nn.CrossEntropyLoss()
        mse_loss=nn.MSELoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-3)
        schedulerG = MultiStepLR(optimizer, milestones=[100,150,200],gamma=0.1)
        if task>=1:
            for epoch in range(args.total_epoch):
                train(train_loader,epoch,task,modelm,task_model)
                test(test_loader,task,modelm,task_model)
                schedulerG.step()

        acc1=load_model(task,modelm)
        task_model.append(copy.deepcopy(modelm))

    task_acc.append(acc1)
    print('Task: '+str(task)+'  Test_accuracy: '+ str(acc1))

# This evaluation only check the accuracy of the current task with all the previous task.
# therefore not correct since if trained the model for the 10 task the 1st task will get 
# evaluated with 10 task. Next cell provides the model evaluation

print(task_acc)


Training Task :---0

Epoch: 0
[Train: ], [0/250: ], [Accuracy: 43.080000], [Loss: 1.374101] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 29.200000], [Loss: 1.987051] [Correct: 1.000000]
Saving..

Epoch: 1
[Train: ], [1/250: ], [Accuracy: 62.320000], [Loss: 0.979397] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 65.200000], [Loss: 1.147452] [Correct: 1.000000]
Saving..

Epoch: 2
[Train: ], [2/250: ], [Accuracy: 66.160000], [Loss: 0.877255] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 65.600000], [Loss: 1.175135] [Correct: 1.000000]
Saving..

Epoch: 3
[Train: ], [3/250: ], [Accuracy: 70.040000], [Loss: 0.810759] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 69.200000], [Loss: 1.028985] [Correct: 1.000000]
Saving..

Epoch: 4
[Train: ], [4/250: ], [Accuracy: 71.280000], [Loss: 0.777010] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 71.000000], [Loss: 1.019032] [Correct: 1.000000]
Saving..

Epoch: 5
[Train: ], [5/250: ], [Accuracy: 74.040000], [Loss: 0.689018] [LossS: 0.000

[Train: ], [47/250: ], [Accuracy: 95.080000], [Loss: 0.151810] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 79.200000], [Loss: 0.979742] [Correct: 1.000000]

Epoch: 48
[Train: ], [48/250: ], [Accuracy: 94.480000], [Loss: 0.149059] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 80.600000], [Loss: 0.804828] [Correct: 1.000000]

Epoch: 49
[Train: ], [49/250: ], [Accuracy: 94.840000], [Loss: 0.144860] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 80.800000], [Loss: 0.848016] [Correct: 1.000000]

Epoch: 50
[Train: ], [50/250: ], [Accuracy: 95.040000], [Loss: 0.145533] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 82.200000], [Loss: 0.817619] [Correct: 1.000000]

Epoch: 51
[Train: ], [51/250: ], [Accuracy: 95.560000], [Loss: 0.131871] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 80.200000], [Loss: 0.892073] [Correct: 1.000000]

Epoch: 52
[Train: ], [52/250: ], [Accuracy: 92.600000], [Loss: 0.207015] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 77.200000], [Loss: 1.030452]

[Test Accuracy: 85.600000], [Loss: 0.708209] [Correct: 1.000000]
Saving..

Epoch: 95
[Train: ], [95/250: ], [Accuracy: 96.280000], [Loss: 0.089555] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 81.200000], [Loss: 0.948586] [Correct: 1.000000]

Epoch: 96
[Train: ], [96/250: ], [Accuracy: 96.760000], [Loss: 0.095851] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 83.400000], [Loss: 0.799787] [Correct: 1.000000]

Epoch: 97
[Train: ], [97/250: ], [Accuracy: 97.640000], [Loss: 0.079798] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 82.000000], [Loss: 0.963764] [Correct: 1.000000]

Epoch: 98
[Train: ], [98/250: ], [Accuracy: 97.200000], [Loss: 0.094522] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 80.800000], [Loss: 0.890782] [Correct: 1.000000]

Epoch: 99
[Train: ], [99/250: ], [Accuracy: 96.920000], [Loss: 0.091517] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 81.000000], [Loss: 0.961804] [Correct: 1.000000]

Epoch: 100
[Train: ], [100/250: ], [Accuracy: 97.960000], [Loss: 

[Test Accuracy: 87.800000], [Loss: 0.582903] [Correct: 1.000000]

Epoch: 142
[Train: ], [142/250: ], [Accuracy: 100.000000], [Loss: 0.005370] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 87.600000], [Loss: 0.584654] [Correct: 1.000000]

Epoch: 143
[Train: ], [143/250: ], [Accuracy: 99.960000], [Loss: 0.006665] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 87.800000], [Loss: 0.578669] [Correct: 1.000000]

Epoch: 144
[Train: ], [144/250: ], [Accuracy: 100.000000], [Loss: 0.005334] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 88.000000], [Loss: 0.576360] [Correct: 1.000000]

Epoch: 145
[Train: ], [145/250: ], [Accuracy: 99.960000], [Loss: 0.005146] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 88.000000], [Loss: 0.573813] [Correct: 1.000000]

Epoch: 146
[Train: ], [146/250: ], [Accuracy: 99.960000], [Loss: 0.005486] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 88.000000], [Loss: 0.576590] [Correct: 1.000000]

Epoch: 147
[Train: ], [147/250: ], [Accuracy: 100.000000], [Lo

[Test Accuracy: 87.800000], [Loss: 0.573186] [Correct: 1.000000]

Epoch: 189
[Train: ], [189/250: ], [Accuracy: 100.000000], [Loss: 0.005422] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 88.000000], [Loss: 0.577615] [Correct: 1.000000]

Epoch: 190
[Train: ], [190/250: ], [Accuracy: 100.000000], [Loss: 0.005357] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 87.800000], [Loss: 0.578393] [Correct: 1.000000]

Epoch: 191
[Train: ], [191/250: ], [Accuracy: 100.000000], [Loss: 0.003920] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 87.800000], [Loss: 0.574735] [Correct: 1.000000]

Epoch: 192
[Train: ], [192/250: ], [Accuracy: 100.000000], [Loss: 0.005006] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 88.200000], [Loss: 0.572670] [Correct: 1.000000]

Epoch: 193
[Train: ], [193/250: ], [Accuracy: 100.000000], [Loss: 0.004761] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 87.800000], [Loss: 0.576899] [Correct: 1.000000]

Epoch: 194
[Train: ], [194/250: ], [Accuracy: 99.960000], [

...

[Train: ], [21/250: ], [Accuracy: 87.960000], [Loss: 0.399575] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 12.400000], [Loss: 0.193307] [Correct: 0.126000]

Epoch: 22
[Train: ], [22/250: ], [Accuracy: 89.080000], [Loss: 0.361757] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 21.800000], [Loss: 0.078504] [Correct: 0.220000]
Saving..

Epoch: 23
[Train: ], [23/250: ], [Accuracy: 88.440000], [Loss: 0.354559] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 12.000000], [Loss: 0.001960] [Correct: 0.120000]

Epoch: 24
[Train: ], [24/250: ], [Accuracy: 89.240000], [Loss: 0.350427] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 15.400000], [Loss: 0.119977] [Correct: 0.156000]

Epoch: 25
[Train: ], [25/250: ], [Accuracy: 90.080000], [Loss: 0.330134] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 17.000000], [Loss: 0.000667] [Correct: 0.170000]

Epoch: 26
[Train: ], [26/250: ], [Accuracy: 88.520000], [Loss: 0.361770] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 14.600000], [Loss: 

[Test Accuracy: 33.000000], [Loss: 0.475940] [Correct: 0.344000]
Saving..

Epoch: 69
[Train: ], [69/250: ], [Accuracy: 94.280000], [Loss: 0.208044] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 32.200000], [Loss: 0.112810] [Correct: 0.326000]

Epoch: 70
[Train: ], [70/250: ], [Accuracy: 94.440000], [Loss: 0.215246] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 27.600000], [Loss: 0.000675] [Correct: 0.276000]

Epoch: 71
[Train: ], [71/250: ], [Accuracy: 93.600000], [Loss: 0.209176] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 20.200000], [Loss: 0.551622] [Correct: 0.214000]

Epoch: 72
[Train: ], [72/250: ], [Accuracy: 93.920000], [Loss: 0.203217] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 31.400000], [Loss: 0.374951] [Correct: 0.326000]

Epoch: 73
[Train: ], [73/250: ], [Accuracy: 93.840000], [Loss: 0.209199] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 32.000000], [Loss: 0.327164] [Correct: 0.330000]

Epoch: 74
[Train: ], [74/250: ], [Accuracy: 94.040000], [Loss: 0.

[Train: ], [116/250: ], [Accuracy: 98.440000], [Loss: 0.085820] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 35.200000], [Loss: 0.001787] [Correct: 0.352000]

Epoch: 117
[Train: ], [117/250: ], [Accuracy: 98.880000], [Loss: 0.083783] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 37.400000], [Loss: 0.001099] [Correct: 0.374000]

Epoch: 118
[Train: ], [118/250: ], [Accuracy: 99.040000], [Loss: 0.080072] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 37.200000], [Loss: 0.044333] [Correct: 0.374000]

Epoch: 119
[Train: ], [119/250: ], [Accuracy: 98.320000], [Loss: 0.090642] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 39.000000], [Loss: 0.043463] [Correct: 0.392000]
Saving..

Epoch: 120
[Train: ], [120/250: ], [Accuracy: 99.280000], [Loss: 0.070397] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 38.200000], [Loss: 0.001072] [Correct: 0.382000]

Epoch: 121
[Train: ], [121/250: ], [Accuracy: 98.920000], [Loss: 0.077518] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 37.40000

[Train: ], [163/250: ], [Accuracy: 99.240000], [Loss: 0.064973] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.800000], [Loss: 0.039088] [Correct: 0.410000]

Epoch: 164
[Train: ], [164/250: ], [Accuracy: 99.200000], [Loss: 0.064243] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.400000], [Loss: 0.079554] [Correct: 0.408000]

Epoch: 165
[Train: ], [165/250: ], [Accuracy: 99.440000], [Loss: 0.061745] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.400000], [Loss: 0.039711] [Correct: 0.406000]

Epoch: 166
[Train: ], [166/250: ], [Accuracy: 99.520000], [Loss: 0.058554] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.800000], [Loss: 0.038131] [Correct: 0.410000]

Epoch: 167
[Train: ], [167/250: ], [Accuracy: 99.440000], [Loss: 0.063351] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.600000], [Loss: 0.037875] [Correct: 0.408000]

Epoch: 168
[Train: ], [168/250: ], [Accuracy: 99.360000], [Loss: 0.057949] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 40.400000], [Loss

[Test Accuracy: 41.200000], [Loss: 0.001404] [Correct: 0.412000]

Epoch: 211
[Train: ], [211/250: ], [Accuracy: 99.520000], [Loss: 0.056117] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 40.200000], [Loss: 0.039054] [Correct: 0.404000]

Epoch: 212
[Train: ], [212/250: ], [Accuracy: 99.560000], [Loss: 0.055179] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 39.800000], [Loss: 0.039843] [Correct: 0.400000]

Epoch: 213
[Train: ], [213/250: ], [Accuracy: 98.960000], [Loss: 0.065578] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 40.400000], [Loss: 0.001459] [Correct: 0.404000]

Epoch: 214
[Train: ], [214/250: ], [Accuracy: 99.560000], [Loss: 0.056110] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 39.400000], [Loss: 0.037350] [Correct: 0.396000]

Epoch: 215
[Train: ], [215/250: ], [Accuracy: 99.480000], [Loss: 0.058736] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 39.400000], [Loss: 0.038226] [Correct: 0.396000]

Epoch: 216
[Train: ], [216/250: ], [Accuracy: 99.360000], [Loss:

In [ ]:
def inference_check_task(dtask,mtask,inputs,task_model):
    joint_entropy=[]
    with torch.no_grad():
        for m in range(mtask+1):
            task_model[m].eval()
            outputs,_=task_model[m](inputs)

            sout=F.softmax(outputs,1)
            entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
            joint_entropy.append(entropy)


    all_entropy=torch.zeros([joint_entropy[0].shape[0],mtask+1]).cuda()
    for i in range(mtask+1):
        all_entropy[:,i]=joint_entropy[i]
    ctask=torch.argmin(all_entropy, axis=1)==dtask
    correct=sum(ctask)
    
    return ctask, correct,all_entropy



def inferecne(test_loader,dtask,mtask,task_model):
    global best_acc
    model=task_model[dtask].eval()
    test_loss = 0
    correct = 0
    total = 0
    cl_loss=0
    tcorrect=0
    accuracy=[]
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets1 = inputs.cuda(), targets.cuda()
            targets=targets1-dtask*args.class_per_task
                        
            if mtask>0:
                correct_sample,Ncorrect,_=inference_check_task(dtask,mtask,inputs,task_model)
                tcorrect+=Ncorrect
                inputs=inputs[correct_sample]
                targets=targets[correct_sample]
                
            if inputs.shape[0]!=0:
                outputs,_ = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()
            total += targets1.size(0)
    if mtask>0:
        taskC= tcorrect.item()/total
    else:
        taskC=1.0 
    acc = 100.*correct/total
    print("[Test Accuracy: %f], [Loss: %f] [Correct: %f]" %(acc,test_loss/(batch_idx+1),taskC))
    
    return acc,taskC

In [ ]:
cl_acc=[]
criterion = nn.CrossEntropyLoss()
for mtask in range(args.num_task):
    accuracy=[]
    corr=[]
    print('#######################################################')
    ttask=mtask+1
    for dtask in range(ttask):
        print('Testing Task :---'+str(dtask))
        test_loader = task_data[dtask][1]
        acc,correct=inferecne(test_loader,dtask,mtask,task_model)
        accuracy.append(acc)
        corr.append(correct)
    task_acc=np.mean(accuracy)
    task_correct=np.mean(corr)
    print('acc: '+str(task_acc)+'  Correct:  '+str(task_correct))
    cl_acc.append(task_acc)
print(np.array(cl_acc))